In [ ]:
import mne
import matplotlib.pyplot as plt
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score
from mne.decoding import CSP
from glob import glob
from scipy.signal import welch
from mpl_toolkits.axes_grid1 import make_axes_locatable

from sklearn.metrics import accuracy_score

from torch.utils.data import TensorDataset, DataLoader
import torch.utils.data as data_utils
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.models import LinearAxis, Range1d

torch.set_default_tensor_type(torch.FloatTensor)
import scipy.io
import torch.utils.data as Data
import torch.nn as nn
import torch.nn.functional as F

In [169]:
result_score = []
name = []
i = 1
fname = 'BCICIV_2a_gdf/A0'+str(i)+'T.gdf'
print(fname)
# path = "BCICIV_2a_gdf/"
# filename = "A01T"

raw = mne.io.read_raw_gdf(fname, eog=['EOG-left', 'EOG-central', 'EOG-right'], preload = True)
raw.drop_channels(['EOG-left', 'EOG-central', 'EOG-right'])

mapping={
'EEG-Fz' : 'Fz',
'EEG-0' : 'FC3',
'EEG-1' : 'FC1',
'EEG-2' : 'FCz',
'EEG-3' : 'FC2',
'EEG-4' : 'FC4',
'EEG-5' : 'C5',
'EEG-C3' : 'C3',
'EEG-6' : 'C1',
'EEG-Cz' : 'Cz',
'EEG-7' : 'C2',
'EEG-C4' : 'C4',
'EEG-8' : 'C6',
'EEG-9' : 'CP3',
'EEG-10' : 'CP1',
'EEG-11' : 'CPz',
'EEG-12' : 'CP2',
'EEG-13' : 'CP4',
'EEG-14' : 'P1',
'EEG-Pz' : 'Pz',
'EEG-15' : 'P2',
'EEG-16' : 'POz',
}

raw.rename_channels(mapping)
montage = mne.channels.make_standard_montage('standard_1020')
raw.set_montage(montage, on_missing='warn')
picks = mne.pick_types(raw.info,eeg=True)
raw.filter(8,35, picks=picks, method='iir', n_jobs=1, iir_params=dict(order=5, ftype='butter', output='sos'), verbose=False)

evetnt_id = {}
if fname == 'BCICIV_2a_gdf/A04T.gdf':
    event_id = dict(left=5, right=6, foot=7, tongue=8)
    event_id_rest = dict(left_rest=5, right_rest=6, foot_rest=7, tongue_rest=8)
else:
    event_id = dict(left=7, right=8, foot=9, tongue=10)
    event_id_rest = dict(left_rest=7, right_rest=8, foot_rest=9, tongue_rest=10)
# picks = mne.pick_types(raw.info,eeg=True)
events, t = mne.events_from_annotations(raw, verbose=False)
print(events)
print(t)
print(event_id)
# events = events[events[:,2] == 7]
# i = 0
tmin = 0.3 # minmax[0:,0][i-1]
tmax = 5.295 # minmax[0:,1][i-1]

epochs = mne.Epochs(raw, events, event_id, tmin=tmin, tmax=tmax, proj=False, picks=picks, baseline=None, preload=True)
# epochs = mne.Epochs(raw, events, {'during' : 1}, -2, -0.5, proj=False,
#                     picks=picks, baseline=None, preload=True, verbose=False)
# print(epochs)
epochs_train = epochs.copy()
# print(epochs_train)
y = epochs.events[:, -1]
# print('y', y, y.shape)
X = epochs.get_data()
# print('x', X, X.shape)
X.shape


# evoked_0.plot_joint()
# evoked_1.plot_joint()
# evoked_2.plot_joint()
# evoked_3.plot_joint()  

BCICIV_2a_gdf/A01T.gdf
Extracting EDF parameters from /Users/germankucheravenko/Documents/Магистратура «Технологии интеграции данных и приложений»/Проекты/EEG/EEGNet-master/BCICIV_2a_gdf/A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG
Creating raw.info structure...
Reading 0 ... 672527  =      0.000 ...  2690.108 secs...


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/mne/io/edf/edf.py:1123: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/contextlib.py:142: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


[[     0      0      5]
 [     0      0      3]
 [ 29683      0      5]
 ...
 [670550      0      6]
 [670550      0      1]
 [671050      0      7]]
{'1023': 1, '1072': 2, '276': 3, '277': 4, '32766': 5, '768': 6, '769': 7, '770': 8, '771': 9, '772': 10}
{'left': 7, 'right': 8, 'foot': 9, 'tongue': 10}
Not setting metadata
288 matching events found
No baseline correction applied
Using data from preloaded Raw for 288 events and 1250 original time points ...
0 bad epochs dropped


(288, 22, 1250)

In [171]:
X1 = X.astype('float32')
X1.shape

(288, 22, 1250)

In [125]:
X1 = X.astype('float')

In [104]:
# Xget = np.random.rand(288, 22, 1250).astype('float32') # np.random.rand generates between [0, 1)
# yget = np.round(np.random.rand(288)) # binary data, so we round it to 0 or 1.

In [156]:
batch_size = 20


from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X1, y1)
print(X_train.shape, y_train.shape)


from torch.utils.data import TensorDataset, DataLoader
import torch.utils.data as data_utils

trans_train = transforms.Compose([transforms.ToTensor()]) # , transforms.Normalize(1.6767584353936251e-09, 5.070522318910306e-06)])
trans_test = transforms.Compose([transforms.ToTensor()]) # , transforms.Normalize(1.9543e-10, 1.1041e-06)])


inputs_train  = torch.tensor(X_train)
print(inputs_train.shape)
targets_train = torch.IntTensor(y_train)

dataset_train = TensorDataset(inputs_train, targets_train)
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

inputs_test  = torch.tensor(X_test)
print(inputs_test.shape)
targets_test = torch.IntTensor(y_test)

dataset_test = TensorDataset(inputs_test, targets_test)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

(216, 22, 1250) (216,)
torch.Size([216, 22, 1250])
torch.Size([72, 22, 1250])


In [ ]:
# Define the network
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(
                in_channels=22,
                out_channels=22,
                kernel_size=200,# Ceramic nuclear 3 * 3
                stride=1,#  
                padding=1# Edge padding
            ),                               #     ((20,16,840) -> (20, 16, 840)
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)      # Dimension Transformation (20, 16, 840) -> (20, 16, 420)
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(
                in_channels=22,
                out_channels=32,
                kernel_size=3,
                stride=1,
                padding=1
            ),                               #     ((20, 16, 420) -> (20, 32, 420)
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)      #     ((20, 32, 420) -> (20, 32, 210)
        )
        self.conv3 = nn.Sequential(
            nn.Conv1d(
                in_channels=32,
                out_channels=48,
                kernel_size=3,
                stride=1,
                padding=1
            ),                               #     ((20, 16, 420) -> (20, 32, 420)
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)      #     ((20, 32, 420) -> (20, 32, 210)
        )
        self.conv4 = nn.Sequential(
            nn.Conv1d(
                in_channels=48,
                out_channels=48,
                kernel_size=3,
                stride=1,
                padding=1
            ),                               #     ((20, 16, 420) -> (20, 32, 420)
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=5)      #     ((20, 32, 420) -> (20, 32, 210)
        )
        self.output = nn.Linear(1248,4)    #   32 * 210 is a number, does not mean that two-dimensional 32 dimensions are multiplied by 210, and the training data changes, this must be changed.
        
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        # out.view Type to one-dimensional, similar to Flatten in Keras
        out = out.view(out.size(0),-1)      #Pytorch's flattening operation is defined in Forward, where the dimension here is 2D 20 * (32 * 210)
        out = self.output(out)
        return out

cnn = CNN()
print(cnn)# Print View Network Structure



CNN(
  (conv1): Sequential(
    (0): Conv1d(22, 22, kernel_size=(200,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv1d(22, 32, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv1d(32, 48, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv4): Sequential(
    (0): Conv1d(48, 48, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
  )
  (output): Linear(in_features=1248, out_features=4, bias=True)
)


In [ ]:
# Preparing to train
optimizer = torch.optim.Adam(cnn.parameters(),lr=0.01,)# Use the ADAM optimization algorithm to train
loss_func = nn.CrossEntropyLoss()#              
for epoch in range(100):
    for step ,(b_x,b_y) in enumerate(train_loader):
#         print(torch.Tensor(b_x).float().dtype)
        output = cnn(b_x)
        #b_y must be a long type
        # print(b_y)
        loss = loss_func(output,b_y.long())

        optimizer.zero_grad()#  
        loss.backward()
        optimizer.step()# Parameter update

        if step%20 ==0:
            test_output = cnn(torch.tensor(X_test))
#             print(test_output.shape)
            # .max (a, 1) Returns the maximum value of each row, [1] .data.NumpY (), return the maximum position index
            _, pred_y = torch.max(test_output, 1)
#             print('pred',pred_y.shape)
            pred_y = pred_y.reshape(len(pred_y),1)
            y_test1 = y_test.reshape(len(y_test),1)
#             print(y_test1.shape)
#             print(pred_y.shape)
            score = accuracy_score(y_test1, pred_y)
            print('Точность на тесте', round(score * 100),'%')
            


            # accuracy = float((pred_y == np.array(y_test).astype(int).sum()) / float(y_test.size(0)))
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy())
            print('finish training')
            torch.save(cnn,'cnn_minist.pkl')# Save training good network

In [271]:
# # Using training good network prediction
# cnn = torch.load('cnn_minist.pkl')

# test_output = cnn(test_x[:20])
# pred_y = torch.max(test_output, 1)[1].data.numpy()

# print(pred_y, 'prediction number')
# print(test_y[:20].numpy(), 'real number')

# test_output1 = cnn(test_x)
# pred_y1 = torch.max(test_output1, 1)[1].data.numpy()
# accuracy = float((pred_y1 == test_y.data.numpy()).astype(int).sum()) / float(test_y.size(0))
# print('accuracy',accuracy)

